In [ ]:
import cv2
import numpy as np

def count_leaves(image_path, min_contour_area=100):
    # Load image
    image = cv2.imread(image_path)

    if image is None:
        print(f"Error: Could not load image from path: {image_path}")
        return

    image = cv2.resize(image, (600, 600))  # Resize for consistency
    
    # Convert to HSV color space
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    
    # Define refined green color range
    lower_green = np.array([35, 40, 40])  # Adjust for better detection
    upper_green = np.array([85, 255, 255])
    mask = cv2.inRange(hsv, lower_green, upper_green)

    # Define a kernel for erosion and apply it
    kernel = np.ones((5, 5), np.uint8)
    mask = cv2.erode(mask, kernel, iterations=3)  # Erode to remove small noise

    # Apply closing (dilation followed by erosion) to merge broken leaf parts
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel, iterations=3)

    # Find contours
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Filter contours based on minimum area to remove noise
    valid_contours = [cnt for cnt in contours if cv2.contourArea(cnt) > min_contour_area]

    leaf_count = len(valid_contours)

    # Draw contours on the original image
    for cnt in valid_contours:
        x, y, w, h = cv2.boundingRect(cnt)
        cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)  # Green box around detected leaves

    # Display leaf count on image
    cv2.putText(image, f'Leaves: {leaf_count}', (20, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    # Show images
    cv2.imshow("Leaf Count", image)
    cv2.imshow("Green Mask (After Erosion)", mask)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    print(f'Total Leaves Counted: {leaf_count}')
    return leaf_count

# Set image path and parameters
image_path = r"C:\Users\MALINGA\Desktop\Images\Plant_Images\Day10_Evening\Health\Plant1_Day10_Evening_Top.jpg"
min_area = 150  # Increase minimum contour area to merge divided leaves

count_leaves(image_path, min_contour_area=min_area)
